# Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  4. Task 4: Evaluating our Pipeline with Ragas
  5. Task 6: Making Adjustments and Re-Evaluating

But first! Let's set some dependencies!

## Dependencies and API Keys:

> NOTE: Please skip the pip install commands if you are running the notebook locally.

In [1]:
#!pip install -qU ragas==0.2.10

In [3]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

We'll also need to provide our API keys.

First, OpenAI's for our LLM/embedding model combination!

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")
os.environ["RAGAS_APP_TOKEN"] = getpass("Please enter your RAGAS Token!")


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [2]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
!curl https://www.myglobalviewpoint.com/route-66-road-trip-itinerary/ -o data/itinerary1.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  686k    0  686k    0     0   472k      0 --:--:--  0:00:01 --:--:--  472k


In [6]:
!curl https://route66roadmap.com/route-66-first-half-itinerary-illinois-to-texas/ -o data/il-to-tx-rt-66.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  389k    0  389k    0     0   256k      0 --:--:--  0:00:01 --:--:--  256k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [9]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

In [10]:
docs

[Document(metadata={'source': 'data/itinerary1.html'}, page_content="The Ultimate Route 66 Road Trip Itinerary (Updated 2025)\n\nJon Miksis\n\nJon Miksis\n\nclock\n\nUpdated January 17, 2025\n\ntourism\n\nDream Destinations\n\nby Jen January 17, 2025\n\nThe Ultimate Route 66 Road Trip Itinerary\n\n242\n\nThis post may have affiliate links, where I may receive a commission if you purchase through them. Here's our Disclosure and Privacy Policy for more info.\n\nImagine the open road ahead, vintage diners flashing neon lights, and the sun setting over the endless horizon. This is Route 66—America’s most iconic road trip. Spanning over 2,400 miles from Chicago to Santa Monica, Route 66 is packed with history, quirky roadside attractions, and stunning landscapes. Whether you’re a seasoned traveler or a newbie to road tripping, this Route 66 road trip guide will help you make the most of every mile.\n\nI’ve broken down the itinerary into manageable chunks, highlighting the best sights and st

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/21 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,As a Route 66 road trip enthusiast seeking to ...,"[Days 1-2: Chicago, Illinois Kick off your Rou...","In Illinois, your Route 66 adventure begins in...",single_hop_specifc_query_synthesizer
1,What should a Route 66 road trip enthusiast kn...,"[known for its unique mineral rock formations,...","Between Springfield and Tulsa on Route 66, Mia...",single_hop_specifc_query_synthesizer
2,What are the top Route 66 attractions to visit...,[The Ultimate Route 66 Road Trip Itinerary (Up...,Top Route 66 attractions in Oklahoma include B...,single_hop_specifc_query_synthesizer
3,what all cool stuff i gotta see in texas on ro...,"[Days 13-14: Amarillo, Texas Get ready to expe...","In Texas, specially Amarillo, you gotta check ...",single_hop_specifc_query_synthesizer
4,What makes Petrified Forest National Park a mu...,[to visit during off-peak times for a quieter ...,"Petrified Forest National Park, located in nor...",single_hop_specifc_query_synthesizer
5,What are the top attractions to visit in St. L...,[<1-hop>\n\na Route 66 road trip in 7 days? Ye...,"According to the sample Route 66 itinerary, St...",multi_hop_specific_query_synthesizer
6,If I want to fully experience the Texas portio...,[<1-hop>\n\nThe Ultimate Route 66 Road Trip It...,To fully experience the Texas portion of Route...,multi_hop_specific_query_synthesizer
7,if i wanna see best of missouri on route 66 tr...,[<1-hop>\n\nThe Ultimate Route 66 Road Trip It...,if you wanna see missouri at its best on route...,multi_hop_specific_query_synthesizer
8,If I want to visit Flagstaff during a Route 66...,[<1-hop>\n\nto visit during off-peak times for...,Flagstaff is a highlight on a Route 66 road tr...,multi_hop_specific_query_synthesizer
9,How can a Route 66 traveler make the most of t...,[<1-hop>\n\nto visit during off-peak times for...,To make the most of a visit to Santa Fe while ...,multi_hop_specific_query_synthesizer


#### OPTIONAL:

If you've provided your Ragas API key - you can use this web interface to look at the created data!

In [14]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/ab2ca549-2d02-42a6-a953-02e27a2b01f5


'https://app.ragas.io/dashboard/alignment/testset/ab2ca549-2d02-42a6-a953-02e27a2b01f5'

## LangChain RAG

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [11]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

#### ❓ Question: 

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

is used to specify the number of characters that should overlap between consecutive chunks of text.  this is useful when the meaning of a sentence or paragraph might depend on the preceding text.

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [14]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [15]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [16]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

Now we can produce a node for retrieval!

In [17]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### Augmented

Let's create a simple RAG prompt!

In [18]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-mini` to avoid using the same model as our judge model.

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [20]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [21]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [22]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [23]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [ ]:
response["response"]

## Evaluating the App with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [25]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [ ]:
dataset.to_pandas()

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [27]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [30]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [ ]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

## Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!

> NOTE: This will be using Cohere's Rerank model (which was updated fairly [recently](https://docs.cohere.com/v2/changelog/rerank-v3.5)) - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

In [32]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

In [34]:
#!pip install -qU cohere langchain_cohere


We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [33]:
retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [34]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [35]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question" : "How are LLM agents useful?"})
response["response"]

In [37]:
import time

for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [40]:
evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [ ]:
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

#### ❓ Question: 

Which system performed better, on what metrics, and why?

1) {'context_recall': 0.7460, 'faithfulness': 0.7891, 'factual_correctness': 0.5809, 'answer_relevancy': 0.8918, 'context_entity_recall': 0.4860, 'noise_sensitivity_relevant': 0.3180}
2) {'context_recall': 0.7133, 'faithfulness': 0.8300, 'factual_correctness': 0.6317, 'answer_relevancy': 0.8740, 'context_entity_recall': 0.4754, 'noise_sensitivity_relevant': 0.2349}

The second evaluation run demonstrated overall improvement compared to the first.

Faithfulness (+5%) and Factual Correctness (+5%) both increased, indicating stronger alignment with the original context and fewer inaccuracies.

Noise Sensitivity improved significantly (lower is better), meaning the model was less distracted by irrelevant information.

Minor declines were observed in Context Recall (-3%) and Context Entity Recall (-1%), suggesting slightly less thorough extraction of details.

Answer Relevancy remained high across both runs, with only a negligible decrease.

Conclusion:
The second evaluation run is more reliable for real-world use, offering greater accuracy and resilience to noise, despite small trade-offs in detail recall.